In [1]:
from shared.utils import load_mnist
import numpy as np

In [2]:
X_train, y_train = load_mnist('./mnist/', kind='train')

In [3]:
X_test, y_test = load_mnist('./mnist/', kind='t10k')

In [4]:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)
X_train_centered = (X_train - mean_vals) / std_val
X_test_centered = (X_test - mean_vals) / std_val

In [5]:
print(X_train_centered.shape, y_train.shape)
print(X_test_centered.shape, y_test.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


In [6]:
import tensorflow as tf

In [7]:
n_features = X_train_centered.shape[1]
n_classes = 10
random_seed = 123
np.random.seed(random_seed)

In [8]:
g = tf.Graph()
with g.as_default():
    tf.set_random_seed(random_seed)
    tf_x = tf.placeholder(dtype=tf.float32, shape=(None, n_features), name='tf_x')
    tf_y = tf.placeholder(dtype=tf.int32, shape=None, name='tf_y')
    y_onehot = tf.one_hot(indices=tf_y, depth=n_classes)
    h1 = tf.layers.dense(inputs=tf_x, units=50, activation=tf.tanh,name='layer1')
    h2 = tf.layers.dense(inputs=h1, units=50, activation=tf.tanh, name='layer2')
    logits = tf.layers.dense(inputs=h2, units=10, activation=None, name='layer3')
    predictions = {
        'classes' : tf.argmax(logits, axis=1, name='predicted_classes'),
        'probabilities' : tf.nn.softmax(logits, name='softmax_tensor')
    }
with g.as_default():
    cost = tf.losses.softmax_cross_entropy(onehot_labels=y_onehot, logits=logits)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(loss=cost)
    init_op = tf.global_variables_initializer()

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [9]:
def create_batch_generator(X, y, batch_size=128, shuffle=False):
    X_copy = np.array(X)
    y_copy = np.array(y)
    if shuffle:
        data = np.column_stack((X_copy, y_copy))
        np.random.shuffle(data)
        X_copy = data[:, :-1]
        y_copy = data[:, -1].astype(int)
    for i in range(0, X.shape[0], batch_size):
        yield (X_copy[i:i+batch_size, :], y_copy[i:i+batch_size])

In [10]:
sess = tf.Session(graph=g)

In [11]:
sess.run(init_op)

In [12]:
from time import time
start = time()
for epoch in range(50):
    training_costs = []
    batch_generator = create_batch_generator(X_train_centered, y_train, batch_size=64)
    for batch_X, batch_y in batch_generator:
        # prepare a dict to feed data to our network:
        feed = {tf_x:batch_X, tf_y:batch_y}
        _, batch_cost = sess.run([train_op, cost], feed_dict=feed)
        training_costs.append(batch_cost)
    print(' -- Epoch %2d Avg. Training Loss: %.4f' % (epoch+1, np.mean(training_costs)))
print('===Time elasped: %ds' % (time() - start))

 -- Epoch  1 Avg. Training Loss: 1.5573
 -- Epoch  2 Avg. Training Loss: 0.9492
 -- Epoch  3 Avg. Training Loss: 0.7499
 -- Epoch  4 Avg. Training Loss: 0.6387
 -- Epoch  5 Avg. Training Loss: 0.5668
 -- Epoch  6 Avg. Training Loss: 0.5160
 -- Epoch  7 Avg. Training Loss: 0.4781
 -- Epoch  8 Avg. Training Loss: 0.4486
 -- Epoch  9 Avg. Training Loss: 0.4247
 -- Epoch 10 Avg. Training Loss: 0.4051
 -- Epoch 11 Avg. Training Loss: 0.3884
 -- Epoch 12 Avg. Training Loss: 0.3741
 -- Epoch 13 Avg. Training Loss: 0.3617
 -- Epoch 14 Avg. Training Loss: 0.3507
 -- Epoch 15 Avg. Training Loss: 0.3408
 -- Epoch 16 Avg. Training Loss: 0.3320
 -- Epoch 17 Avg. Training Loss: 0.3239
 -- Epoch 18 Avg. Training Loss: 0.3165
 -- Epoch 19 Avg. Training Loss: 0.3097
 -- Epoch 20 Avg. Training Loss: 0.3035
 -- Epoch 21 Avg. Training Loss: 0.2976
 -- Epoch 22 Avg. Training Loss: 0.2921
 -- Epoch 23 Avg. Training Loss: 0.2870
 -- Epoch 24 Avg. Training Loss: 0.2822
 -- Epoch 25 Avg. Training Loss: 0.2776


In [13]:
feed = {tf_x: X_test_centered}
y_pred = sess.run(predictions['classes'], feed_dict=feed)
print('Test Accuracy: %.2f%%' % (100*np.sum(y_pred == y_test)/y_test.shape[0]))

Test Accuracy: 93.89%
